In [18]:
%load_ext autoreload
%autoreload 2

In [19]:
import math

import pandas as pd
import numpy as np

import lass.datasets
from lass.log_handling import LogLoader

## Load data

Load all data from logs into a single dataframe.

Sorry memory.

For one model family, e.g. BIG-G T=0, with all sizes, shots, and tasks, this requires about 20GB of memory and takes about 5 minutes to load on the server we used for experiments.

In [3]:
loader = LogLoader(
    logdir='../artifacts/logs',
    tasks='paper-full',
    model_families=['BIG-G T=0'],
    # model_sizes=['128b'],
    query_types=['multiple_choice'],
    # shots=[0],
    include_unknown_shots=True,
    exclude_faulty_tasks=True,
)

In [4]:
bigdf = lass.datasets.to_dataframe(loader)

Some extra columns that we will use a lot

In [5]:
bigdf['n_targets'] = bigdf['targets'].map(lambda x: len(x))
bigdf['conf_normalized'] = bigdf['normalized_scores'].map(lambda s: math.exp(np.max(s)))
bigdf['conf_absolute']= bigdf['absolute_scores'].map(lambda s: math.exp(np.max(s)))

Results for a single model and single number of shots.
This should contain all the testing instances exactly once.

In [6]:
example = bigdf.query('model_family == "BIG-G T=0" & model_name == "128b" & shots == 0')
len(example)

55431

## Basic Stats

In [7]:
df_n_instances = bigdf.groupby(['model_name', 'shots']).size()

print(f"Total #tasks: {bigdf['task'].nunique()}")
print(f"Total #unique instances: {df_n_instances.max()}")
print(f"Total #records: {len(bigdf)}")
print(f"Equal #unique instances per shot and model: {bigdf.groupby(['task', 'model_name', 'shots']).size().groupby(['task']).nunique().nunique() == 1}")
print(f"Different shots: {df_n_instances.index.levels[1].values}") # type: ignore
print(f"Different models: {df_n_instances.index.levels[0].values}") # type: ignore




Total #tasks: 118
Total #unique instances: 55431
Total #records: 2660688
Equal #unique instances per shot and model: True
Different shots: [0 1 2 3]
Different models: ['125m' '128b' '16m' '1b' '244m' '27b' '2b' '2m' '422m' '4b' '53m' '8b']


## Trivia

### Faulty Logging

The scores for the context_definition_alignment where not logged properly in the BIG-G logs. All scores are 0. The aggregate scores currently reported in the plots in the repo make more sense.

In [8]:
bigdf.query('task == "context_definition_alignment"')['correct'].value_counts()

Series([], Name: correct, dtype: int64)

### Faulty targets

There are multiple choice questions with only 1 option. It's likely a bug tho, ass all the scores are 0, and this is the only task with a mismatch between `targets` and `target_values` columns.

All answers are wrong.

In [9]:
example[example['n_targets'] == 1].groupby('task')[["n_targets", "correct"]].value_counts()


task        n_targets  correct
arithmetic  1          0.0        992
dtype: int64

Even though there is sometimes high confidence in this single target.

In [10]:
example[example['n_targets'] == 1]['conf_absolute'].max()

0.37418268939347205

It's the only one task with difference in length between the `targets` and `target_values` field.

In [11]:
example[example['n_targets'] != example['target_values'].map(lambda x: len(x))]['task'].unique()

array(['arithmetic'], dtype=object)

But all instances seem copies of valid queries.

In [12]:
example.query('task == "arithmetic"')['n_targets'].value_counts()

1    992
7    797
6    171
5     24
Name: n_targets, dtype: int64

There are multiple choice questions with 100 options. This one seems fine.

In [13]:
example[example['n_targets'] == 100][["n_targets", "correct"]].value_counts()

n_targets  correct
100        0.0        279
           1.0         62
dtype: int64

## Varia

In [14]:
df_accs = (bigdf
    .groupby(['task', 'model_name', 'shots'])
    .agg(
        acc=('correct', 'mean'), 
        random_acc=('n_targets', lambda x: (1/x).mean()),
        n_targets=('n_targets', 'mean'),
    )  # type: ignore
)
df_accs['rel_acc_improvement'] = (df_accs['acc'] - df_accs['random_acc']) / (1 - df_accs['random_acc'])
df_accs['acc_improvement'] = (df_accs['acc'] - df_accs['random_acc'])
df_accs_best = (df_accs
    .xs(0, level='shots', drop_level=False)
    .xs('128b', level='model_name', drop_level=False)
    .sort_values('acc', ascending=False))
df_accs_best.to_csv('accs_0.csv')



# acc: 0.75, random acc: 0.25 -> 2/3 | (0.75 - 0.25) / (1 - 0.25) = 1

In [15]:
# df_accs_best = pd.read_csv('../artifacts/csv-datasets/accs_3.csv')
df_accs_best.describe()

,acc,random_acc,n_targets,rel_acc_improvement,acc_improvement
count,118.000000,118.000000,118.000000,118.000000,118.000000
mean,0.376433,0.304463,5.926796,0.091950,0.071970
std,0.160381,0.148706,9.644268,0.208893,0.134129
min,0.004883,0.011905,2.000000,-1.136492,-0.483950
25%,0.245636,0.200000,2.000000,-0.007426,-0.003713
50%,0.381406,0.250000,4.000000,0.069409,0.051515
75%,0.500000,0.500000,5.000000,0.212257,0.137213
max,0.864865,0.574172,84.000000,0.729730,0.426852
